In [126]:
import os
from dotenv import load_dotenv

In [127]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [128]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-lite-preview-02-05',
    temperature=0.3
)

In [129]:
a = llm.invoke("hi how are you")
print(a.content)

I am doing well, thank you for asking! How are you today?


In [130]:
import pandas as pd
df = pd.read_csv('transaction_2025.csv')

In [131]:
df.head()

,Date,Description,Amount (INR),Payment Method
0,2025-02-10,McDonald's,4283.83,Credit Card
1,2024-03-06,Hotstar,1399.45,UPI
2,2024-11-12,Big Bazaar,4569.77,UPI
3,2024-11-16,Metro Ticket,2050.67,UPI
4,2024-06-07,Mobile Recharge,954.59,Debit Card


In [132]:
unique_transactions = df['Description'].unique()

array(['Hotstar', 'Big Bazaar', 'Metro Ticket', 'Mobile Recharge',
       'Medicine Purchase', 'Udemy Course', 'Gift Purchase',
       'Fixed Deposit', 'Swiggy Order'], dtype=object)

In [133]:
def hop(start,stop,step):
    for i in range(start,stop,step):
        yield i
    yield stop
index_list = list(hop(0,len(unique_transactions),30))
index_list

[0, 30, 41]

In [134]:
def categorize_transactions(transaction_names,llm):
    response = (llm.invoke("Can you categories the data of expenses with the appropriate catogory for example food , travel,entertainment and etc but the order should be like Food - Swiggy , the whole should be in this form giving (do not group them)..."+ "," + transaction_names +"Give the output in the format of Category - Transactions ")).content
    response = response.split('\n')
    cleaned_data = [item.replace("*", "") for item in response]
    for item in cleaned_data:
        print(item)
    categories_df = pd.DataFrame({'Transaction vs category': cleaned_data})
    categories_df[['Category', 'Transaction']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df
    

In [135]:
categories_df_all = pd.DataFrame()
for i in range(0,len(index_list)-1):
    transaction_name = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_name = ','.join(transaction_name)
    
    categories_df = categorize_transactions(transaction_name,llm)
    categories_df_all = pd.concat([categories_df_all,categories_df],ignore_index=True)

Here's the categorized list of expenses in the requested format:

   Food - McDonald's
   Entertainment - Hotstar
   Shopping - Big Bazaar
   Travel - Metro Ticket
   Communication - Mobile Recharge
   Health - Medicine Purchase
   Education - Udemy Course
   Gifts - Gift Purchase
   Investments - Fixed Deposit
   Food - Swiggy Order
   Health - Gym Membership
   Entertainment - Concert Pass
   Charity - Charity
   Charity - Temple Donation
   Utilities - Water Bill
   Travel - Train Ticket
   Shopping - Book Purchase
   Food - Reliance Fresh
   Food - Local Kirana
   Education - College Fees
   Unplanned - Unplanned Expense
   Shopping - Amazon Purchase
   Food - Local Market
   Education - Coaching Fee
   Travel - Ola Ride
   Travel - Petrol Pump
   Investments - Stock Purchase
   Investments - Mutual Fund SIP
   Utilities - Wi-Fi Bill
   Food - Domino's
Here's the categorization of your expenses:

   Food - Swiggy
   Food - Zomato Order
   Healthcare - Doctor Visit
   Healthcare - P

In [136]:
categories_df_all

,Transaction vs category,Category,Transaction
0,Here's the categorized list of expenses in the...,Here's the categorized list of expenses in the...,None
1,,,None
2,Food - McDonald's,Food,McDonald's
3,Entertainment - Hotstar,Entertainment,Hotstar
4,Shopping - Big Bazaar,Shopping,Big Bazaar
5,Travel - Metro Ticket,Travel,Metro Ticket
6,Communication - Mobile Recharge,Communication,Mobile Recharge
7,Health - Medicine Purchase,Health,Medicine Purchase
8,Education - Udemy Course,Education,Udemy Course
9,Gifts - Gift Purchase,Gifts,Gift Purchase


In [137]:
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Category,Transaction
0,Here's the categorized list of expenses in the...,Here's the categorized list of expenses in the...,None
1,,,None
2,Food - McDonald's,Food,McDonald's
3,Entertainment - Hotstar,Entertainment,Hotstar
4,Shopping - Big Bazaar,Shopping,Big Bazaar
5,Travel - Metro Ticket,Travel,Metro Ticket
6,Communication - Mobile Recharge,Communication,Mobile Recharge
7,Health - Medicine Purchase,Health,Medicine Purchase
8,Education - Udemy Course,Education,Udemy Course
9,Gifts - Gift Purchase,Gifts,Gift Purchase


In [138]:
df_new = df
df_new = pd.merge(df_new, categories_df_all, left_on='Description', right_on='Transaction', how='left')
df_new = df_new.drop(columns=['Transaction vs category','Transaction'])

In [139]:
df_new

,Date,Description,Amount (INR),Payment Method,Category
0,2025-02-10,McDonald's,4283.83,Credit Card,Food
1,2024-03-06,Hotstar,1399.45,UPI,Entertainment
2,2024-11-12,Big Bazaar,4569.77,UPI,Shopping
3,2024-11-16,Metro Ticket,2050.67,UPI,Travel
4,2024-06-07,Mobile Recharge,954.59,Debit Card,Communication
...,...,...,...,...,...
224,2024-12-28,Hotstar,703.08,UPI,Entertainment
225,2025-02-06,Temple Donation,1945.02,UPI,Charity
226,2024-05-29,Stock Purchase,2172.96,Credit Card,Investments
227,2024-02-27,Mobile Recharge,4683.61,Credit Card,Communication
